**There were some issues with the Actors data that were parsed. Wrong date formats, wrong names , and The use of 'N/A' to indicate null. What the following script does is clean up the dates of birth and dates of death and it sets empty dob to "1700-00-00"**

In [ ]:
import csv
import re
from datetime import datetime
import dateutil.parser as dparser
import pandas as pd
from io import StringIO
from google.colab import files
import os

def clean_date(date_str):
    """Extract and standardize date from various formats."""
    if pd.isna(date_str) or date_str == 'N/A':
        return None

    # Remove age and location information
    date_str = re.sub(r'\(age \d+[–\-]\d+\)', '', date_str)
    date_str = re.sub(r'\(aged \d+[–\-]\d+\)', '', date_str)

    # Try to extract just the date part for processing
    date_match = re.search(r'\d{4}[-/]?\d{1,2}[-/]?\d{1,2}', date_str)
    if date_match:
        try:
            parsed_date = datetime.strptime(date_match.group(), '%Y-%m-%d')
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            pass

    # Try another approach for year-only dates
    year_match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if year_match:
        return f"{year_match.group()}-01-01"

    # For more complex cases, try dateutil parser
    try:
        date_text = re.sub(r'[^0-9\-\/\s,\.]+', ' ', date_str)
        parsed_date = dparser.parse(date_text, fuzzy=True)
        return parsed_date.strftime('%Y-%m-%d')
    except:
        return None

def clean_country(country_str):
    """Standardize country names with enhanced cleaning."""
    if pd.isna(country_str) or country_str == 'N/A':
        return None

    # Remove parentheses and their contents
    country_str = re.sub(r'\(.*?\)', '', country_str)

    # Remove any bracketed numbers/annotations
    country_str = re.sub(r'\[.*?\]', '', country_str)

    # Standardize common variations
    country_map = {
        'US': 'United States',
        'U.S.': 'United States',
        'USA': 'United States',
        'United States': 'United States',
        'England': 'United Kingdom',
        'Wales': 'United Kingdom',
        'Pennsylvania': 'United States',
        'Russian Empire': 'Russia',
        'German Empire': 'Germany',
        'Austria-Hungary': 'Austria'
    }

    # Clean punctuation and whitespace
    country_str = re.sub(r'[^\w\s]', '', country_str).strip()

    # Split on comma if present
    country_part = country_str.split(',')[0] if ',' in country_str else country_str

    return country_map.get(country_part, country_part)

def is_valid_person(name, dob):
    """Check if entry is a valid person (not a company or studio)."""
    if pd.isna(name) or name == 'N/A':
        return False

    # List of non-person entries
    non_people = [
        'Warner Bros.', 'Columbia', 'Metro-Goldwyn-Mayer',
        'Universal', 'RKO Radio', 'Greek', 'English', 'Arabic'
    ]

    # Check for studio names
    if re.search(r'\bstudio\b', name, flags=re.IGNORECASE):
        return False

    return name not in non_people

def clean_name(name_str):
    """Clean and normalize person names but keep dots."""
    if pd.isna(name_str):
        return None

    # Remove numbers and special characters except apostrophes, hyphens, spaces, and dots
    name_str = re.sub(r"[^a-zA-Z\s\.'\-]", '', name_str)

    # Remove extra whitespace
    return ' '.join(name_str.split()).title()

def normalize_csv_data(input_data):
    """Process the CSV data and return normalized data."""
    df = pd.read_csv(input_data, header=None,
                     names=['Name', 'DOB', 'Country', 'DOD'])

    output_data = []

    for _, row in df.iterrows():
        # Clean name first
        raw_name = row['Name']
        clean_name_val = clean_name(raw_name)

        if not is_valid_person(clean_name_val, row['DOB']):
            continue

        # Clean other fields
        dob_clean = clean_date(row['DOB']) or "1700-01-01"
        country_clean = clean_country(row['Country'])
        dod_clean = clean_date(row['DOD'])

        output_data.append({
            'Name': clean_name_val,
            'DOB': dob_clean,
            'Country': country_clean,
            'DOD': dod_clean
        })

    output_df = pd.DataFrame(output_data)
    return output_df.drop_duplicates(subset=['Name', 'DOB'])


def main():
    print("CSV Normalizer for Person Database")
    print("==================================")
    print("1. Upload your CSV file")

    # For Google Colab: Upload file
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
        return

    # Process the first uploaded file
    file_name = list(uploaded.keys())[0]
    print(f"\nProcessing file: {file_name}")

    # Normalize the data
    normalized_df = normalize_csv_data(file_name)

    # Show sample of normalized data
    print("\nSample of normalized data:")
    print(normalized_df.head())

    # Statistics about the data
    print(f"\nTotal records: {len(normalized_df)}")
    print(f"Missing DOB values (set to 1900-01-01): {normalized_df[normalized_df['DOB'] == '1900-01-01'].shape[0]}")
    print(f"Missing Country values: {normalized_df['Country'].isna().sum()}")
    print(f"Missing DOD values: {normalized_df['DOD'].isna().sum()}")

    # Save to CSV
    output_filename = f"normalized_{file_name}"
    normalized_df.to_csv(output_filename, index=False)
    print(f"\nSaved normalized data to {output_filename}")

    # For Google Colab: Download the file
    print("\nDownloading normalized CSV file...")
    files.download(output_filename)

if __name__ == "__main__":
    # Install required packages if needed
    try:
        import dateutil
    except ImportError:
        print("Installing required packages...")
        !pip install python-dateutil
        import dateutil

    main()

CSV Normalizer for Person Database
1. Upload your CSV file


Saving normalized_normalized_actors (2) (1).csv to normalized_normalized_actors (2) (1).csv

Processing file: normalized_normalized_actors (2) (1).csv

Sample of normalized data:
               Name         DOB        Country         DOD
0              Name  1700-01-01        Country        None
1      Henry Blanke  1901-12-30        Germany  1981-05-28
2       Leo Mccarey  1898-10-03  United States  1969-07-05
3  Louis D. Lighton  1895-11-25  United States  1963-02-01
4    Samuel Goldwyn  1882-08-27         Russia  1974-01-31

Total records: 6656
Missing DOB values (set to 1900-01-01): 0
Missing Country values: 1209
Missing DOD values: 3803

Saved normalized data to normalized_normalized_normalized_actors (2) (1).csv



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**This script is to normalise the country column in actors**

In [ ]:
import pandas as pd
import re

# List of valid modern countries
VALID_COUNTRIES = {
    'United States', 'Canada', 'Mexico', 'United Kingdom', 'France', 'Germany', 'Italy', 'Spain',
    'Portugal', 'Switzerland', 'Belgium', 'Netherlands', 'Austria', 'Poland', 'Russia', 'Ukraine',
    'Belarus', 'Czech Republic', 'Slovakia', 'Hungary', 'Romania', 'Bulgaria', 'Serbia', 'Croatia',
    'Slovenia', 'Bosnia and Herzegovina', 'Montenegro', 'North Macedonia', 'Albania', 'Greece',
    'Turkey', 'Cyprus', 'Ireland', 'Iceland', 'Norway', 'Sweden', 'Finland', 'Denmark', 'Estonia',
    'Latvia', 'Lithuania', 'Moldova', 'China', 'Japan', 'South Korea', 'North Korea', 'India',
    'Pakistan', 'Bangladesh', 'Sri Lanka', 'Nepal', 'Bhutan', 'Myanmar', 'Thailand', 'Vietnam',
    'Cambodia', 'Laos', 'Malaysia', 'Singapore', 'Indonesia', 'Philippines', 'Australia', 'New Zealand',
    'Papua New Guinea', 'Brazil', 'Argentina', 'Chile', 'Peru', 'Colombia', 'Venezuela', 'Ecuador',
    'Bolivia', 'Paraguay', 'Uruguay', 'Guyana', 'Suriname', 'Egypt', 'Tunisia', 'Algeria', 'Morocco',
    'Libya', 'Sudan', 'South Sudan', 'Ethiopia', 'Somalia', 'Kenya', 'Uganda', 'Tanzania', 'Rwanda',
    'Burundi', 'South Africa', 'Namibia', 'Botswana', 'Zimbabwe', 'Zambia', 'Malawi', 'Mozambique',
    'Angola', 'Palestine', 'Lebanon', 'Jordan', 'Syria', 'Iraq', 'Iran', 'Saudi Arabia', 'Kuwait',
    'Bahrain', 'Qatar', 'United Arab Emirates', 'Oman', 'Yemen', 'Afghanistan', 'Tajikistan',
    'Uzbekistan', 'Turkmenistan', 'Kyrgyzstan', 'Kazakhstan', 'Mongolia', 'Taiwan', 'Hong Kong'
}

# US states that should be mapped to United States
US_STATES = {
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
    'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
}

# US state abbreviations
US_STATE_ABBR = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
    'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
    'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
}

# Historical entities and their modern equivalents
HISTORICAL_ENTITIES = {
    'Kingdom of France': 'France',
    'French Algeria': 'Algeria',
    'Empire of Japan': 'Japan',
    'Grand Duchy of Baden': 'Germany',
    'AustriaHungary': 'Austria',
    'Second Polish Republic': 'Poland',
    'South Vietnam': 'Vietnam',
    'Republic of Venice': 'Italy',
    'Mandatory Palestine': 'Palestine',
    'Israel':'Palestine',
    'Imperial State of Iran': 'Iran',
    'United Arab Republic': 'Egypt',
    'South African Republic': 'South Africa',
    'Saarland': 'Germany',
    'Territory of Hawaii': 'United States',
    'British India': 'India',
    'New South Wales': 'Australia',
    'Kingdom of Yugoslavia': 'Serbia',
    'Second French Empire': 'France',
    'West Germany': 'Germany',
    'Cape Colony': 'South Africa',
    'Nazi Germany': 'Germany',
    'Germanoccupied Netherlands': 'Netherlands',
    'under Spanish occupation': 'Spain',
    'SFR Yugoslavia': 'Serbia',
    'British Palestine': 'Palestine',
    'French Indochina': 'Vietnam',
    'Dahomey': 'Benin',
    'FPR Yugoslavia': 'Serbia',
    'Russian Empire': 'Russia',
    'Colony of New South Wales': 'Australia',
    'Ottoman Empire': 'Turkey',
    'Czechoslovakia': 'Czech Republic',
    'Territory of the Saar Basin': 'Germany',
    'Duchy of Warsaw': 'Poland',
    'Sultanate of Zanzibar': 'Tanzania',
    'Russian SFSR': 'Russia',
    'BadenWürttemberg': 'Germany',
    'Union of South Africa': 'South Africa',
    'Ukrainian SSR': 'Ukraine',
    'AustroHungarian Empire': 'Austria',
    'Mandate Palestine': 'Palestine',
    'Weimar Republic': 'Germany',
    'Ottoman Turkey': 'Turkey',
    'Qing Empire': 'China',
    'Uganda Protectorate': 'Uganda',
    'Orange Free State': 'South Africa',
    'Qing China': 'China',
    'Kingdom of Romania': 'Romania',
    'East Germany': 'Germany',
    'British America': 'United States',
    'Colony of South Australia': 'Australia',
    'Kingdom of Egypt': 'Egypt',
    'Kingdom of Greece': 'Greece',
    'Siam': 'Thailand',
    'Germanoccupied Poland': 'Poland',
    'South Australia': 'Australia',
    'Irish Free State': 'Ireland',
    'Roman Republic': 'Italy',
    'Austrian Empire': 'Austria',
    'Soviet Union': 'Russia',
    'Federation of Malaya': 'Malaysia',
    'USSR': 'Russia',
    'Catalonia': 'Spain',
    'Roman Empire': 'Italy',
    'modernday Bosnia and Herzegovina': 'Bosnia and Herzegovina',
    'British Empire': 'United Kingdom',
    'RuandaUrundi': 'Rwanda',
    'Polish Peoples Republic': 'Poland',
    'Holy Roman Empire': 'Germany',
    'Glamorgan': 'United Kingdom',
    'Austrian Poland': 'Poland',
    'Kingdom of England': 'United Kingdom',
    'Pahlavi Iran': 'Iran',
    'Nazioccupied Serbia': 'Serbia',
    'Alliedoccupied Germany': 'Germany',
    'Kingdom of Prussia': 'Germany',
    'German Reich': 'Germany',
    'Far Eastern Republic': 'Russia',
    'UK': 'United Kingdom',
    'USA': 'United States',
    'US': 'United States'
}

# Cities and regions with their corresponding countries
CITIES_TO_COUNTRIES = {
    'New York City': 'United States',
    'Los Angeles': 'United States',
    'Chicago': 'United States',
    'London': 'United Kingdom',
    'Paris': 'France',
    'Rome': 'Italy',
    'Toronto': 'Canada',
    'Montreal': 'Canada',
    'Quebec': 'Canada',
    'Jerusalem': 'Palestine',
    'Hong Kong': 'Hong Kong',
    'Moscow': 'Russia',
    'Edinburgh': 'United Kingdom',
    'Oxford': 'United Kingdom',
    'Jaffa': 'Palestine',
    'Aleppo': 'Syria',
    'Porto': 'Portugal',
    'Rotterdam': 'Netherlands',
    'Singapore': 'Singapore',
    'Rustington': 'United Kingdom',
    'Buenos Aires': 'Argentina',
    'Kingston upon Hull': 'United Kingdom',
    'Hounslow': 'United Kingdom',
    'Prechistoye': 'Russia',
    'Saddleworth England': 'United Kingdom',
    'Prague': 'Czech Republic',
    'Straubing': 'Germany',
    'East Sussex': 'United Kingdom',
    'Corsica': 'France',
    'Yorkshire': 'United Kingdom',
    'Salzburg': 'Austria',
    'Washington DC': 'United States',
    'Monaco': 'Monaco',
    'Ciudad Ayala': 'Mexico',
    'British Hong Kong': 'Hong Kong',
    'Massalia': 'France',
    'Aomori Prefecture': 'Japan',
    'Kagoshima City': 'Japan',
    'Ontario': 'Canada',
    'British Columbia': 'Canada',
    'southern Korea': 'South Korea'
}

def normalize_country(country):
    """
    Normalize country name or return empty string if invalid
    """
    if not country or pd.isna(country):
        return ''

    country = str(country).strip()

    # Check if already a valid country
    if country in VALID_COUNTRIES:
        return country

    # Check if it's a US state
    if country in US_STATES:
        return 'United States'

    # Check if it's a US state abbreviation
    if country in US_STATE_ABBR:
        return 'United States'

    # Check if it's a historical entity
    if country in HISTORICAL_ENTITIES:
        return HISTORICAL_ENTITIES[country]

    # Check if it's a city
    if country in CITIES_TO_COUNTRIES:
        return CITIES_TO_COUNTRIES[country]

    # If we couldn't normalize it, return empty string
    return ''

def clean_countries_in_csv(input_file, output_file):
    """
    Clean country data in a CSV file using pandas
    """
    # Read the CSV file
    try:
        df = pd.read_csv(input_file)
    except pd.errors.EmptyDataError:
        # Handle empty file case
        print(f"Error: {input_file} is empty or not a valid CSV")
        return
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    # Check if 'Country' column exists
    if 'Country' not in df.columns:
        print("Error: CSV file does not contain a 'Country' column")
        return

    # Keep track of statistics
    total_rows = len(df)
    valid_countries_count = 0
    empty_countries_count = 0
    normalized_countries_count = 0
    countries_set_to_empty = 0

    # Make a copy of the original Country column for comparison
    df['OriginalCountry'] = df['Country'].copy()

    # Apply country normalization
    df['Country'] = df['Country'].apply(normalize_country)

    # Count statistics
    for idx, row in df.iterrows():
        if pd.isna(row['OriginalCountry']) or row['OriginalCountry'] == '':
            empty_countries_count += 1
        elif row['Country'] == '':
            countries_set_to_empty += 1
        elif row['Country'] != row['OriginalCountry']:
            normalized_countries_count += 1
        else:
            valid_countries_count += 1

    # Remove the temporary column used for comparison
    df = df.drop('OriginalCountry', axis=1)

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

    # Print statistics
    print(f"CSV Processing Complete:")
    print(f"- Total rows processed: {total_rows}")
    print(f"- Originally valid countries: {valid_countries_count}")
    print(f"- Originally empty countries: {empty_countries_count}")
    print(f"- Normalized countries: {normalized_countries_count}")
    print(f"- Set to empty due to invalid values: {countries_set_to_empty}")
    print(f"- Output saved to: {output_file}")

if __name__ == "__main__":
    input_file = "actors.csv"  # Replace with your input file path
    output_file = "cleaned_countries.csv"  # Replace with your output file path
    clean_countries_in_csv(input_file, output_file)

CSV Processing Complete:
- Total rows processed: 6655
- Originally valid countries: 4378
- Originally empty countries: 1209
- Normalized countries: 454
- Set to empty due to invalid values: 614
- Output saved to: cleaned_countries.csv


**There were issues with release date similar to actor dob column. This script Fixes the faulty release dates setting unknown values to '1700-01-01'. In addition. The runtime column used to include some text that was falsely parsed.**

In [ ]:
import re
from datetime import datetime
import dateutil.parser as dparser
import pandas as pd

def clean_date(date_str):
    """Extract and standardize date from various formats."""
    if pd.isna(date_str) or date_str == 'N/A':
        return f'1700-01-01'

    # Remove parentheses and their contents
    date_str = re.sub(r'\([^)]*\)', '', str(date_str)).strip()

    # Standardize date formats
    try:
        parsed_date = dparser.parse(date_str, fuzzy=True)
        return parsed_date.strftime('%Y-%m-%d')
    except ValueError:
        try:
            year = int(date_str)
            return f'{year}-01-01'  # Default to January 1st if only year
        except ValueError:
            return f'1700-01-01'

def clean_production_company(company_str):
    """Clean production company names."""
    if pd.isna(company_str) or company_str == 'N/A':
        return None

    # Remove extra characters and corporation types
    company_str = re.sub(r'[,."]+', '', str(company_str))
    company_str = re.sub(r'\b(Pictures|Corporation|Inc|The)\b', '', company_str, flags=re.IGNORECASE)

    # Clean up whitespace
    company_str = company_str.strip()

    return company_str

def clean_runtime(runtime_str):
    """Clean and convert runtime to integer."""
    if pd.isna(runtime_str) or runtime_str == 'N/A':
        return '0'

    try:
        runtime = int(runtime_str)
        return runtime
    except ValueError:
        return '0'

def normalize_movies_data(input_file):
    """Process the movie data from CSV and return normalized data."""
    df = pd.read_csv(input_file)

    # Clean each column
    df['releaseDate'] = df['releaseDate'].apply(clean_date)
    df['productionCompany'] = df['productionCompany'].apply(clean_production_company)
    df['runTime'] = df['runTime'].apply(clean_runtime)

    # Drop rows with missing values in critical columns (e.g., title)
    df.dropna(subset=['Title'], inplace=True)

    return df

def main():
    input_file = 'movies (1).csv'

    # Normalize the data
    normalized_df = normalize_movies_data(input_file)

    # Show sample of normalized data
    print("\nSample of normalized data:")
    print(normalized_df.head())
    print(f"\nTotal records: {len(normalized_df)}")

    # Save to CSV
    output_filename = "cleaned_movies.csv"
    normalized_df.to_csv(output_filename, index=False)
    print(f"\nSaved normalized data to {output_filename}")

if __name__ == "__main__":
    main()


Sample of normalized data:
                    Title releaseDate productionCompany Language runTime
0  The Life of Emile Zola  1937-08-11       Warner Bros  English     116
1         The Awful Truth  1937-10-21          Columbia  English      91
2     Captains Courageous  1937-05-11            Loew's  English     117
3                Dead End  1937-08-27    United Artists  English      93
4          The Good Earth  1937-01-29            Loew's  English     138

Total records: 4116

Saved normalized data to cleaned_movies.csv


Create actors and Movies lists to use in the construction of the nominations table


In [ ]:
import pandas as pd

# Load actors CSV
actors_df = pd.read_csv('actors.csv')

# Create list of dictionaries: [{"Name": name, "DOB": dob}, ...]
actors = [{"Name": row["Name"], "DOB": row["DOB"]} for _, row in actors_df.iterrows()]

# Load movies CSV
movies_df = pd.read_csv('movies.csv')

# Create list of dictionaries: [{"Title": title, "releaseDate": release_date}, ...]
movies = [{"Title": row["Title"], "releaseDate": row["releaseDate"]} for _, row in movies_df.iterrows()]

# Print results
print("Actors:")
print(actors)

print("\nMovies:")
print(movies)

Actors:
[{'Name': 'Henry Blanke', 'DOB': '1901-12-30'}, {'Name': 'Leo Mccarey', 'DOB': '1898-10-03'}, {'Name': 'Louis D. Lighton', 'DOB': '1895-11-25'}, {'Name': 'Samuel Goldwyn', 'DOB': '1882-08-27'}, {'Name': 'Irving Thalberg', 'DOB': '1899-05-30'}, {'Name': 'Albert Lewin', 'DOB': '1894-09-23'}, {'Name': 'Darryl F. Zanuck', 'DOB': '1902-09-05'}, {'Name': 'Frank Capra', 'DOB': '1897-05-18'}, {'Name': 'Charles R. Rogers', 'DOB': '1892-07-15'}, {'Name': 'Joe Pasternak', 'DOB': '1901-09-19'}, {'Name': 'Pandro S. Berman', 'DOB': '1905-03-28'}, {'Name': 'David O. Selznick', 'DOB': '1902-05-10'}, {'Name': 'Sidney Franklin', 'DOB': '1893-03-21'}, {'Name': 'William Dieterle', 'DOB': '1893-07-15'}, {'Name': 'Gregory La Cava', 'DOB': '1892-03-10'}, {'Name': 'William Wellman', 'DOB': '1896-02-29'}, {'Name': 'Spencer Tracy', 'DOB': '1900-04-05'}, {'Name': 'Charles Boyer', 'DOB': '1899-08-28'}, {'Name': 'Napoleon Bonaparte', 'DOB': '1769-08-15'}, {'Name': 'Fredric March', 'DOB': '1897-08-31'}, {'N

This script takes the nomination. Most nominations have the format" \{movie name \} – \{actors names\}' but some nominations flip this order. The script splits it at the "–" to create two parts. It checks if the first part is a valid movie name in which case it will look for it in the movies list and retrieve the release date and adding it to the row, Then it moves onto the second part and splits it at the separators 'and' and ',' removing everything after the word 'for'. If the first part is not a valid movie name then that means that the order is flipped so the first part is treated as the list of the actor names nad the second is the movie name. There are some special fomated nominations such as 'Best International Feature Film' Where it has format \{movie name \} \({Language}\) – \{actors names\}'. and "best Song" and "Best Music" where it has the format '\(\{song name\}\) from \{movie name\}. And the script deals with them accordingly

In [ ]:
import pandas as pd
import re
import logging
import sys
from datetime import datetime
import string

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('nomination_processor.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)




# Normalize text by removing punctuation and extra whitespace
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip().lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text)
    return text

# Clean actor and movie data
actors_df = pd.DataFrame(actors_data)
actors_df['CleanName'] = actors_df['Name'].apply(clean_text)

movies_df = pd.DataFrame(movies_data)
movies_df['CleanTitle'] = movies_df['Title'].apply(clean_text)

def assign_role(category):
    actor_keywords = ["Actor", "Actress", "Leading Role", "Supporting Role"]
    director_keywords = ["Directing", "Assistant Director"]
    writer_keywords = ["Writing", "Screenplay", "Story"]
    composer_keywords = ["Music", "Song", "Score", "Scoring"]
    sound_visual_keywords = ["Sound", "Visual Effects", "Sound Editing", "Sound Mixing"]
    short_doc_keywords = ["Short Film", "Short Subject", "Documentary"]
    design_keywords = ["Production Design", "Costume Design", "Makeup", "Art Direction"]
    cinematography_editing_keywords = ["Cinematography", "Editing"]
    picture_keywords = ["Picture", "Motion Picture", "Production"]
    international_keywords = ["Foreign Language Film", "International Feature Film"]
    vfx_keywords = ["Special Effects"]

    if any(keyword in category for keyword in actor_keywords):
        return "Actor/Actress"
    elif any(keyword in category for keyword in director_keywords):
        return "Director"
    elif any(keyword in category for keyword in writer_keywords):
        return "Writer"
    elif any(keyword in category for keyword in composer_keywords):
        return "Composer"
    elif any(keyword in category for keyword in sound_visual_keywords):
        return "Sound/Visual Engineer"
    elif any(keyword in category for keyword in short_doc_keywords):
        return "Director/Producer"
    elif any(keyword in category for keyword in design_keywords):
        return "Designer"
    elif any(keyword in category for keyword in cinematography_editing_keywords):
        return "Cinematographer/Editor"
    elif any(keyword in category for keyword in picture_keywords):
        return "Producer"
    elif any(keyword in category for keyword in vfx_keywords):
        return "VFX artist"
    elif any(keyword in category for keyword in international_keywords):
        return "Director/Producer"
    else:
        return "Unknown"

def find_movie_by_title(title):
    cleaned = clean_text(title)
    match = movies_df[movies_df['CleanTitle'] == cleaned]
    if not match.empty:
        return match.iloc[0].to_dict()
    return None

def find_actor_by_name(name, nomination=None):
    """Find an actor by name in the actors DataFrame"""
    if not name:
        logger.warning("Empty actor name provided")
        return None

    match = actors_df[actors_df['Name'].str.lower().str.strip() == name.strip().lower()]

    if not match.empty:
        actors = match.to_dict('records')
        return actors[0]  # Automatically return the first match

    logger.warning(f"No actor found with name: {name}")
    return None

def extract_movie_title(nomination, category):
    # Handle International / Foreign category: title is before '('
    if "International Feature" in category or "Foreign Language Film" in category:
        # Remove anything from the first '(' onward, then split on dash to separate from director part
        title_part = re.split(r'\(', nomination)[0]
        title = re.split(r'–|-', title_part)[0].strip()
        return title

    # Music category — extract title after 'from'
    if "Music" in category and "from" in nomination:
        match = re.search(r'from\s+(.*?)(?=\s+[–-])', nomination)
        if match:
            return match.group(1).strip()

    # General case — try both sides of dash
    parts = re.split(r'[–-]', nomination)
    if len(parts) < 2:
        return nomination.strip()

    first_part = parts[0].strip()
    second_part = parts[1].strip()

    if find_movie_by_title(first_part):
        return first_part
    if find_movie_by_title(second_part):
        return second_part

    return first_part  # fallback

def extract_actors(nomination, movie_title, category):
    actor_part = nomination.replace(movie_title, "").replace("–", "").replace("-", "").strip()

    if "International Feature" in category or "Foreign Language Film" in category:
        # Look for "by <name>" or "directed by <name>"
        match = re.search(r'by\s+(.*)', actor_part, re.IGNORECASE)
        if match:
            return [name.strip() for name in re.split(r';|,|\sand\s', match.group(1))]
        return []

    if " for " in actor_part:
        actor_part = actor_part.split(" for ")[0].strip()

    if " as " in actor_part:
        actor_part = actor_part.split(" as ")[0].strip()

    if "Music" in category or "Song" in category:
        if "Music by" in actor_part:
            music_by_match = re.search(r'Music by\s+(.*?)(?:;|$|\sLyrics by)', actor_part)
            lyrics_by_match = re.search(r'Lyrics by\s+(.*?)(?:;|$)', actor_part)
            composers = []
            if music_by_match:
                composers.extend([name.strip() for name in re.split(r';|,|\sand\s', music_by_match.group(1))])
            if lyrics_by_match:
                composers.extend([name.strip() for name in re.split(r';|,|\sand\s', lyrics_by_match.group(1))])
            return composers
        by_match = re.search(r'by\s+(.*?)(?:;|$)', actor_part)
        if by_match:
            return [name.strip() for name in re.split(r';|,|\sand\s', by_match.group(1))]
        return [name.strip() for name in re.split(r';|,|\sand\s', actor_part)]

    return [name.strip() for name in re.split(r';|,|\sand\s', actor_part)]

def process_nominations(input_file, output_file):
    logger.info(f"Processing nominations from {input_file}")
    try:
        nominations_df = pd.read_csv(input_file)
    except Exception as e:
        logger.error(f"Error reading input file: {str(e)}")
        return None

    result_data = []

    for idx, row in nominations_df.iterrows():
        category = row['Category']
        nomination = row['Nomination']

        movie_title = extract_movie_title(nomination, category)
        movie = find_movie_by_title(movie_title)
        role = assign_role(category)
        actors_list = extract_actors(nomination, movie_title, category)

        for actor_name in actors_list:
            cleaned_actor_name = actor_name.strip().strip(string.punctuation)
            actor_info = find_actor_by_name(cleaned_actor_name, nomination)

            result_data.append({
                'iteration': row['iteration'],
                'Category': category,
                'Nomination': nomination,
                'Won': row['Won'],
                'Movie': movie['Title'] if movie else movie_title,
                'Movie Release Date': movie['releaseDate'] if movie else "",
                'Actor': actor_name,
                'Actor DOB': actor_info['DOB'] if actor_info else "",
                'Role': role
            })

    result_df = pd.DataFrame(result_data)
    try:
        result_df.to_csv(output_file, index=False)
    except Exception as e:
        logger.error(f"Error writing output file: {str(e)}")

    return result_df

def main():
    logger.info("Nomination processor started")
    input_file = input("Enter the path to the nominations CSV file: ")
    output_file = input("Enter the path for the output CSV file: ")

    try:
        df = process_nominations(input_file, output_file)
        if df is not None:
            print(f"Processing completed. {len(df)} entries written to {output_file}")
    except Exception as e:
        logger.error(f"Processing failed: {str(e)}")
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Enter the path to the nominations CSV file: nominations.csv
Enter the path for the output CSV file: processed.csv


Streaming output truncated to the last 5000 lines.


Processing completed. 15598 entries written to processed.csv


In this code it cleans up the category column in the nominations table as there were inconsistencies in names. And it rechecks the movie names and actor names.

In [ ]:
# Create lookup maps (case insensitive)
actor_dob_map = {actor['Name'].strip().lower(): actor['DOB'] for actor in actors_data}
movie_date_map = {movie['Title'].strip().lower(): movie['releaseDate'] for movie in movies_data}

# Define swappable categories
swappable_categories = {
    "best actor", "best actress",
    "best actor in a supporting role",
    "best actress in a supporting role"
}

# Normalize function
def normalize(text):
    return str(text).strip().lower()

# Function to fix a row
def fix_row(row):
    category = normalize(row.get('Category', ''))
    actor = str(row.get('Actor', '')).strip()
    movie = str(row.get('Movie', '')).strip()
    actor_dob = str(row.get('Actor DOB', '')).strip()
    movie_date = str(row.get('Movie Release Date', '')).strip()

    actor_norm = normalize(actor)
    movie_norm = normalize(movie)

    # Swap actor and movie if they're likely switched
    if category in swappable_categories:
        if actor_norm in movie_date_map and movie_norm in actor_dob_map:
            actor, movie = movie, actor
            actor_norm, movie_norm = movie_norm, actor_norm
            actor_dob = ''
            movie_date = ''

    # Fill missing Movie Release Date
    if not movie_date and movie_norm in movie_date_map:
        movie_date = movie_date_map[movie_norm]

    # Fill missing Actor DOB
    if not actor_dob and actor_norm in actor_dob_map:
        actor_dob = actor_dob_map[actor_norm]

    # Update the row
    row['Actor'] = actor
    row['Movie'] = movie
    row['Actor DOB'] = actor_dob
    row['Movie Release Date'] = movie_date

    return row

# Load your nominations CSV
df = pd.read_csv('processed.csv')

# Apply fix to all rows
df = df.apply(fix_row, axis=1)

# Filter out rows where either actor or movie not in respective maps
df_filtered = df[df['Actor'].str.lower().isin(actor_dob_map.keys()) &
                 df['Movie'].str.lower().isin(movie_date_map.keys())]

# Save cleaned CSV
df_filtered.to_csv('processed_cleaned.csv', index=False)

# Optionally show the result
print(df_filtered)


       iteration                Category  \
0             10  Outstanding Production   
2             10  Outstanding Production   
3             10  Outstanding Production   
5             10  Outstanding Production   
6             10  Outstanding Production   
...          ...                     ...   
15580         96     Best Visual Effects   
15582         96     Best Visual Effects   
15588         96     Best Visual Effects   
15590         96     Best Visual Effects   
15592         96     Best Visual Effects   

                                              Nomination  Won  \
0      The Awful Truth – Leo McCarey and Everett Risk...    0   
2      Captains Courageous – Louis D. Lighton for Met...    0   
3      Dead End – Samuel Goldwyn and Merritt Hulbert ...    0   
5      The Good Earth – Irving Thalberg and Albert Le...    0   
6      The Good Earth – Irving Thalberg and Albert Le...    0   
...                                                  ...  ...   
15580  Guardians

In [ ]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00


Generate fake data for User and user nominations tables

In [ ]:
from faker import Faker
import pandas as pd
import random

fake = Faker()

def generate_fake_users(n=10):
    users = set()
    emails = set()
    data = []

    while len(data) < n:
        username = fake.user_name()
        email = fake.unique.email()
        gender = random.choice(['Male', 'Female', 'Other', None])
        birthdate = fake.date_of_birth(minimum_age=18, maximum_age=90)
        country = fake.country()

        if username in users or email in emails:
            continue

        users.add(username)
        emails.add(email)

        data.append({
            'username': username,
            'email': email,
            'gender': gender,
            'birthdate': birthdate,
            'country': country
        })

    return pd.DataFrame(data)

# Generate 100 fake users and save to CSV
df = generate_fake_users(100)
df.to_csv('fake_users.csv', index=False)
print("CSV file 'fake_users.csv' has been created.")

CSV file 'fake_users.csv' has been created.


In [ ]:
import pandas as pd
import random

# Attempt to load the nominations CSV with fallback encodings
try:
    nominations_df = pd.read_csv('nomination.csv', delimiter=';', encoding='latin1')
except:
    try:
        nominations_df = pd.read_csv('nomination.csv', delimiter=';', encoding='cp1252')
    except:
        nominations_df = pd.read_csv('nomination.csv', delimiter=';', encoding='latin1',
                                     on_bad_lines='skip', errors='replace')

# Load the users CSV
users_df = pd.read_csv('fake_users.csv')

# Randomly assign usernames to a subset of nominations
sample_size = int(len(nominations_df) * 0.7)
assigned_indices = random.sample(range(len(nominations_df)), sample_size)
user_nomination_df = nominations_df.iloc[assigned_indices].copy()

# Add randomly selected usernames
user_nomination_df['username'] = random.choices(users_df['username'], k=len(user_nomination_df))

# Select only the necessary columns if they exist
columns_to_use = [col for col in ['username', 'movie_title', 'movie_release_date',
                                  'person_name', 'person_dob', 'role', 'category', 'iteration']
                  if col in user_nomination_df.columns]
user_nomination_df = user_nomination_df[columns_to_use]

# Rename columns to match target schema
column_mapping = {
    'movie_title': 'title',
    'person_name': 'full_name'
}
user_nomination_df.rename(columns={k: v for k, v in column_mapping.items() if k in user_nomination_df.columns},
                          inplace=True)

# Save to a new CSV
user_nomination_df.to_csv('user_nominations.csv', index=False, encoding='utf-8')
print("CSV file 'user_nominations.csv' has been created.")



CSV file 'user_nominations.csv' has been created.


In [ ]:
import pandas as pd
import re

# Load the CSV file
df = pd.read_csv('processed_cleaned.csv')  # Replace with your actual file name

# Remove square brackets around numbers in the 'category' column
df['Category'] = df['Category'].str.replace(r'\[(\d+)\]', r'\1', regex=True)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False)

print("Updated CSV saved as 'updated_file.csv'")

Updated CSV saved as 'updated_file.csv'


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('updated_file.csv')  # Replace with your actual file path

# Remove square brackets from numbers in the 'category' column
df['Category'] = df['Category'].str.replace(r'\[(\d+)\]', r'\1', regex=True)

# Select only the 'iteration' and 'category' columns
df_filtered = df[['iteration', 'Category']]

# Remove duplicate rows
df_filtered = df_filtered.drop_duplicates()

# Save the result to a new CSV file
df_filtered.to_csv('iteration_category_unique.csv', index=False)

print("Processed CSV saved as 'iteration_category_unique.csv'")

Processed CSV saved as 'iteration_category_unique.csv'
